In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import pickle
import gc
from tqdm.notebook import tqdm
from collections import deque


In [7]:
# Load the data and stack
test = list(range(11,60))
shop_ids = list(range(2,10)) + list(range(11,20))
X = []
y = []
file_path = "simplified_data/"
for i in tqdm(range(4)):
    with open(file_path+"reshaped_train_data_part_"+str(i)+".pkl",'rb') as f:
        X += pickle.load(f)
    with open(file_path+"reshaped_train_label_data_part_"+str(i)+".pkl",'rb') as f:
        y += pickle.load(f)
X_data = np.concatenate(X)
del X; gc.collect();
y_data = np.concatenate(y)
del y; gc.collect()
    

0

In [8]:
X_data.shape

(13405056, 2, 31)

In [9]:
# Scale the the target variable
y_data = y_data.reshape(-1)
y_data = y_data.clip(0,40)
max_log_y = np.log1p(40)
y_data_scaled = np.log1p(y_data)/max_log_y
del y_data; gc.collect();

(13405056,)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data_scaled,
                                                 test_size=0.2)
del X_data; gc.collect();

In [ ]:
X_train.shape

In [15]:
model_lstm = Sequential()
model_lstm.add(LSTM(256, input_shape=(2,31), return_sequences=True))
# model_lstm.add(LSTM(32, return_sequences=True))
model_lstm.add(LSTM(64))
model_lstm.add(Dense(units=1))
model_lstm.compile(loss='mse', optimizer='adam', metrics=['accuracy']) # Increase Learning rate by steps of 0.1
model_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 2, 256)            294912    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                82176     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 377,153
Trainable params: 377,153
Non-trainable params: 0
_________________________________________________________________


In [17]:
history = model_lstm.fit(X_train, y_train, 
                         validation_data=(X_val, y_val), 
                         epochs=30, 
                         batch_size=10000)

Train on 10724044 samples, validate on 2681012 samples
Epoch 1/30
10724044/10724044 [==============================] - 685s 64us/sample - loss: 0.0055 - accuracy: 0.8895 - val_loss: 0.0055 - val_accuracy: 0.8895
Epoch 2/30
10724044/10724044 [==============================] - 679s 63us/sample - loss: 0.0054 - accuracy: 0.8895 - val_loss: 0.0054 - val_accuracy: 0.8894
Epoch 3/30
10724044/10724044 [==============================] - 675s 63us/sample - loss: 0.0054 - accuracy: 0.8895 - val_loss: 0.0054 - val_accuracy: 0.8895
Epoch 4/30
10724044/10724044 [==============================] - 675s 63us/sample - loss: 0.0053 - accuracy: 0.8895 - val_loss: 0.0053 - val_accuracy: 0.8895
Epoch 5/30
10724044/10724044 [==============================] - 680s 63us/sample - loss: 0.0053 - accuracy: 0.8895 - val_loss: 0.0053 - val_accuracy: 0.8895
Epoch 6/30
10724044/10724044 [==============================] - 676s 63us/sample - loss: 0.0053 - accuracy: 0.8895 - val_loss: 0.0053 - val_accuracy: 0.8895
Epo

In [24]:
result = model_lstm.predict(X_val)

In [26]:
result = np.expm1(result*max_log_y)

In [28]:
y_val_unscaled = np.expm1(y_val * max_log_y)

In [30]:
from sklearn.metrics import mean_squared_error
from numpy import sqrt
rmse = sqrt(mean_squared_error(y_val_unscaled,result))
print('Val RMSE: %.3f' % rmse)

Val RMSE: 0.955


In [23]:
result.min()

-0.29517037

In [31]:
model_lstm.save('LSTM_Model_v1.h5')

In [33]:
model_test = load_model('LSTM_Model_v1.h5')

In [34]:
result_v2 = model_test.predict(X_val)
result_v2 = np.expm1(result_v2*max_log_y)

In [35]:
rmse = sqrt(mean_squared_error(y_val_unscaled,result_v2))
print('Val RMSE: %.3f' % rmse)

Val RMSE: 0.955


In [36]:
del X_train; del X_val; gc.collect();